In [ ]:
import mne_connectivity
import mne
import h5py
import numpy as np
import os
import pandas as pd
from connection_complexity.data.raw_data.EDF.edf_helpers import read_edf

from eeg_prep.preprocess.ieeg import preprocess_ieeg
from mne_connectivity import spectral_connectivity_time

from mne import make_fixed_length_epochs

In [ ]:
# calculate minimum frequency possible
sfreq = 2048
n_times = 2048//2 
n_cycles = 7
dur = float(n_times) / sfreq
cycle_freq = n_cycles / dur
print(cycle_freq)

14.0


In [32]:
mapping_path = "/media/dan/Data/manuscripts/manuiscript_0001_hfo_rates/data/FULL_composite_patient_info.csv"  
ilae_path = "/media/dan/Data/manuscripts/manuiscript_0001_hfo_rates/ravi_hfo_numbers~N59+v03.csv"
bad_channels_path = "/media/dan/Data/manuscripts/manuiscript_0001_hfo_rates/data/bad_ch_review.xlsx"
edf_path = "/media/dan/Data/data/iEEG/raw_ieeg/baseline_patients/baseline_edfs"
output_path = "/media/dan/Data/git/network_miner/connectivity/output"

pid_source_path = "/media/dan/Data/manuscripts/manuiscript_0001_hfo_rates/ravi_hfo_numbers~N59+v03.csv"

In [33]:
# get metadata
mappings = pd.read_csv(mapping_path)
ilae = pd.read_csv(ilae_path)
bad_channels = pd.read_excel(bad_channels_path)
bad_channels["use"] = bad_channels["use"].fillna(1)
bad_channels["use2"] = bad_channels["use2"].fillna(1)
bad_channels["use"] = bad_channels["use"].astype(bool)
bad_channels["use2"] = bad_channels["use2"].astype(bool)

# OR bad_channel columns
bad_channels["bad_channel"] = ~(bad_channels["use"] & bad_channels["use2"])

# for each patient in mappings, find the corresponding ilae number. The patient may not be in the ilae dataset but has a designation of seizureFree or not.
# if the patient is not in the ilae dataset, then use the seizureFree column to determine the ilae number where -1 is seizureFree and 100 is not seizureFree
ilae_numbers = {}
for pid in mappings["pid"].unique():
    if pid in ilae["patient"].values:
        ilae_numbers[pid] = ilae[ilae["patient"] == pid]["ilae"].values[0]
    else:
        if mappings[mappings["pid"] == pid]["seizureFree"].values[0] == True:
            ilae_numbers[pid] = -1
        else:
            ilae_numbers[pid] = 100

# now we have a dictionary of ilae numbers for each patient. Fill in the mappings dataframe with these numbers which has multiple rows for each patient
ilae_list = []
for pid in mappings["pid"]:
    ilae_list.append(ilae_numbers[pid])
mappings["ilae"] = ilae_list


# Perform the merge as before
mappings = mappings.merge(
    bad_channels[['pid', 'ch', 'bad_channel']],
    left_on=['pid', 'electrode'],
    right_on=['pid', 'ch'],
    how='left'
)

# Drop the 'ch' column if needed
mappings = mappings.drop(columns=['ch'])

# Fill NaN values in 'bad_channel' with 0
mappings['bad_channel'] = mappings['bad_channel'].fillna(0)

In [34]:
# get 3 random pids with ilae 1-2 
df = pd.read_csv(pid_source_path)
df = df[df["ilae"] < 3]
df = df.sample(3, random_state=14)
pidsToUse = df['patient'].unique()
pidsToUse

array([34, 91, 47])

In [ ]:

files = os.listdir(edf_path)
for edf_file in files:
    f = os.path.join(edf_path, edf_file)
    pid = int(edf_file.split("_")[0])
    if pid not in pidsToUse:
        continue
    print(f"Processing {f}")
    patient_info = mappings[mappings["pid"] == pid]
    raw = read_edf(f, preload=True)
    sfreq = raw.info["sfreq"]

    raw.info["line_freq"] = 60

    raw = preprocess_ieeg(raw) 

    # remove bad channels
    raw = raw.drop_channels(patient_info[patient_info["bad_channel"] == 1]["electrode"].values)

    # average reference
    raw = raw.set_eeg_reference(ref_channels="average", projection=False, verbose=False)

    # make epochs
    dur_msec = 500
    dur_sec = dur_msec / 1000
    overlap_msec = 0
    overlap_sec = overlap_msec / 100
    epochs = make_fixed_length_epochs(raw, duration=dur_sec,overlap=0, preload=True)
    methods = ['coh',"pli","wpli","plv"]
    freqs = np.arange(14,sfreq//2)
    conn = spectral_connectivity_time(
        epochs.get_data(),
        freqs=freqs,
        method=methods,
        sfreq=sfreq,
        faverage=True,
        verbose=True,
        n_jobs=27,
        # indices=indices,
        mode="multitaper",
    )
    for i,m in enumerate(methods):
        out = conn[i].get_data(output='dense')
        # save to file
        np.save(os.path.join(output_path, f"{pid:03}_{m}_winmsec-{dur_msec:06}_overlap-{overlap_msec:06}.npy"), out)

Processing /media/dan/Data/data/iEEG/raw_ieeg/baseline_patients/baseline_edfs/034_Baseline.EDF
Not setting metadata
606 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 606 events and 1024 original time points ...
0 bad epochs dropped
Fmin was not specified. Using fmin=min(freqs)
Fmax was not specified. Using fmax=max(freqs).
only using indices for lower-triangular matrix
Connectivity computation...
   Processing epoch 1 / 5 ...


[Parallel(n_jobs=27)]: Using backend LokyBackend with 27 concurrent workers.
[Parallel(n_jobs=27)]: Done  45 tasks      | elapsed:    8.6s
[Parallel(n_jobs=27)]: Done  91 out of  97 | elapsed:   14.7s remaining:    1.0s
[Parallel(n_jobs=27)]: Done  97 out of  97 | elapsed:   15.3s finished
[Parallel(n_jobs=27)]: Using backend LokyBackend with 27 concurrent workers.
[Parallel(n_jobs=27)]: Done  45 tasks      | elapsed:    5.5s
[Parallel(n_jobs=27)]: Done 135 tasks      | elapsed:    9.2s
[Parallel(n_jobs=27)]: Done 261 tasks      | elapsed:   14.4s
[Parallel(n_jobs=27)]: Done 423 tasks      | elapsed:   21.2s
[Parallel(n_jobs=27)]: Done 621 tasks      | elapsed:   29.4s
[Parallel(n_jobs=27)]: Done 855 tasks      | elapsed:   39.1s
[Parallel(n_jobs=27)]: Done 1125 tasks      | elapsed:   50.4s
[Parallel(n_jobs=27)]: Done 1431 tasks      | elapsed:  1.1min
[Parallel(n_jobs=27)]: Done 1773 tasks      | elapsed:  1.3min
[Parallel(n_jobs=27)]: Done 2151 tasks      | elapsed:  1.6min
[Paralle

   Processing epoch 2 / 5 ...


[Parallel(n_jobs=27)]: Using backend LokyBackend with 27 concurrent workers.
[Parallel(n_jobs=27)]: Done  45 tasks      | elapsed:    7.9s
[Parallel(n_jobs=27)]: Done  91 out of  97 | elapsed:   13.9s remaining:    0.9s
[Parallel(n_jobs=27)]: Done  97 out of  97 | elapsed:   14.5s finished
[Parallel(n_jobs=27)]: Using backend LokyBackend with 27 concurrent workers.
[Parallel(n_jobs=27)]: Done  45 tasks      | elapsed:    5.3s
[Parallel(n_jobs=27)]: Done 135 tasks      | elapsed:    8.9s
[Parallel(n_jobs=27)]: Done 261 tasks      | elapsed:   13.8s
[Parallel(n_jobs=27)]: Done 423 tasks      | elapsed:   20.2s
[Parallel(n_jobs=27)]: Done 621 tasks      | elapsed:   28.0s
[Parallel(n_jobs=27)]: Done 855 tasks      | elapsed:   37.2s
[Parallel(n_jobs=27)]: Done 1125 tasks      | elapsed:   47.9s
[Parallel(n_jobs=27)]: Done 1431 tasks      | elapsed:   59.9s
[Parallel(n_jobs=27)]: Done 1773 tasks      | elapsed:  1.2min
[Parallel(n_jobs=27)]: Done 2151 tasks      | elapsed:  1.5min
[Paralle

   Processing epoch 3 / 5 ...


...

   Processing epoch 4 / 5 ...


[Parallel(n_jobs=27)]: Using backend LokyBackend with 27 concurrent workers.
[Parallel(n_jobs=27)]: Done  45 tasks      | elapsed:    9.3s
[Parallel(n_jobs=27)]: Done  91 out of  97 | elapsed:   16.4s remaining:    1.1s
[Parallel(n_jobs=27)]: Done  97 out of  97 | elapsed:   17.2s finished
[Parallel(n_jobs=27)]: Using backend LokyBackend with 27 concurrent workers.
[Parallel(n_jobs=27)]: Done  45 tasks      | elapsed:    5.4s
[Parallel(n_jobs=27)]: Done 135 tasks      | elapsed:    9.1s
[Parallel(n_jobs=27)]: Done 261 tasks      | elapsed:   14.1s
[Parallel(n_jobs=27)]: Done 423 tasks      | elapsed:   20.6s
[Parallel(n_jobs=27)]: Done 621 tasks      | elapsed:   28.5s
[Parallel(n_jobs=27)]: Done 855 tasks      | elapsed:   37.9s
[Parallel(n_jobs=27)]: Done 1125 tasks      | elapsed:   48.8s
[Parallel(n_jobs=27)]: Done 1431 tasks      | elapsed:  1.0min
[Parallel(n_jobs=27)]: Done 1773 tasks      | elapsed:  1.2min
[Parallel(n_jobs=27)]: Done 2151 tasks      | elapsed:  1.5min
[Paralle

   Processing epoch 5 / 5 ...


[Parallel(n_jobs=27)]: Using backend LokyBackend with 27 concurrent workers.
[Parallel(n_jobs=27)]: Done  45 tasks      | elapsed:    9.4s
[Parallel(n_jobs=27)]: Done  91 out of  97 | elapsed:   16.6s remaining:    1.1s
[Parallel(n_jobs=27)]: Done  97 out of  97 | elapsed:   17.4s finished
[Parallel(n_jobs=27)]: Using backend LokyBackend with 27 concurrent workers.
[Parallel(n_jobs=27)]: Done  45 tasks      | elapsed:    5.4s
[Parallel(n_jobs=27)]: Done 135 tasks      | elapsed:    9.0s
[Parallel(n_jobs=27)]: Done 261 tasks      | elapsed:   14.0s
[Parallel(n_jobs=27)]: Done 423 tasks      | elapsed:   20.5s
[Parallel(n_jobs=27)]: Done 621 tasks      | elapsed:   28.5s
[Parallel(n_jobs=27)]: Done 855 tasks      | elapsed:   37.8s
[Parallel(n_jobs=27)]: Done 1125 tasks      | elapsed:   48.7s
[Parallel(n_jobs=27)]: Done 1431 tasks      | elapsed:  1.0min
[Parallel(n_jobs=27)]: Done 1773 tasks      | elapsed:  1.2min
[Parallel(n_jobs=27)]: Done 2151 tasks      | elapsed:  1.5min
[Paralle

[Connectivity computation done]
